### Conectamos a la base de datos "integridad.db" y creamos una tabla "Usuarios". 

- La columna ID identifica de manera unica a cada usuario de la tabla, por lo que no puede repetirse. Por lo que utilizaremos la restriccion Primary Key.

- La columna Nombre no puede tener valores nulos por lo que aplicaremos la restricción NOT NULL que asegura que una columna no puede tener valores nulos. Esto significa que siempre debe haber un valor en esa columna

- Email deben ser únicos por lo que utilizaremos "UNIQUE"que asegura que todos los valores en una columna sean únicos. Esto significa que no puede haber valores duplicados en esa columna

In [8]:
import sqlite3

conn = sqlite3.connect('integridad.db')
cursor = conn.cursor()

# Crear tablas con diferentes restricciones
cursor.execute('''
CREATE TABLE IF NOT EXISTS Usuarios (
    ID INTEGER PRIMARY KEY,
    Nombre TEXT NOT NULL,
    Email TEXT UNIQUE
);
''')

##### Creamos una tabla "Productos" donde la columna "Precio" debe tener un valor mayor que 0 por lo que aplicamos la restricción CHECK asegura que los valores en una columna cumplan con una condición específica

In [9]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Productos (
    ID INTEGER PRIMARY KEY,
    Nombre TEXT,
    Precio REAL CHECK (Precio > 0)
);
''')

Creamos una tabla "Pedidos" donde la columna ClienteID debe coincidir con un ID en la tabla Clientes. Aplicamos la restricción FOREIGN KEY asegura que los valores en una columna coincidan con los valores de una columna en otra tabla

In [24]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Pedidos (
    ID INTEGER PRIMARY KEY,
    UsuarioID INTEGER,
    FOREIGN KEY (UsuarioID) REFERENCES Usuarios(ID)
);
''')

#### Insertamos datos en nuestras tablas

In [25]:
# Insertar datos
cursor.execute("INSERT OR IGNORE INTO Productos (ID, Nombre, Precio) VALUES (1, 'Gaseosa', 5000)")
cursor.execute("INSERT OR IGNORE INTO Pedidos (ID, UsuarioID) VALUES (1, 1)") 
cursor.execute("INSERT OR IGNORE INTO Usuarios (ID, Nombre, Email) VALUES (1, 'Gus', 'gus@ejemplo.com')")

-----------------------------------------------------------------------------------------------------------------------------------------------------------------

## OPRIMIZACION DE CONSULTAS

### Consulta ineficiente con SELECT *

In [47]:
print("Consulta ineficiente (SELECT *):")
for row in cursor.execute("SELECT * FROM Usuarios;"):
    print(row)

Consulta ineficiente (SELECT *):
(1, 'Gus', 'gus@ejemplo.com')


### Consulta eficiente seleccionando solo las columnas necesarias

Seleccionar solo las columnas necesarias reduce la cantidad de datos transferidos entre la base de datos y la aplicación, lo que mejora el rendimiento de la consulta y reduce el uso de memoria.

In [13]:
print("\nConsulta eficiente (SELECT columnas específicas):")
for row in cursor.execute("SELECT ID, Nombre, Email FROM Usuarios;"):
    print(row)


Consulta eficiente (SELECT columnas específicas):
(1, 'Gus', 'gus@ejemplo.com')


### Consulta con Subconsulta

In [26]:
print("Consulta con subconsulta:")
for row in cursor.execute('''
SELECT Nombre, (SELECT COUNT(*) FROM Pedidos WHERE Pedidos.UsuarioID = Usuarios.ID) AS TotalPedidos
FROM Usuarios;
'''):
    print(row)



Consulta con subconsulta:
('Gus', 1)


### Consulta con JOIN

Seleccionar solo las columnas necesarias reduce la cantidad de datos transferidos entre la base de datos y la aplicación, lo que mejora el rendimiento de la consulta y reduce el uso de memoria

In [27]:
print("\nConsulta con JOIN:")
for row in cursor.execute('''
SELECT Usuarios.Nombre, COUNT(Pedidos.ID) AS TotalPedidos
FROM Usuarios
LEFT JOIN Pedidos ON Usuarios.ID = Pedidos.UsuarioID
GROUP BY Usuarios.Nombre;
'''):
    print(row)


Consulta con JOIN:
('Gus', 1)


### Indices

### Implementamos indices

Estos índices ayudan a la base de datos a encontrar rápidamente las filas relevantes para los joins, mejorando el tiempo de respuesta de las consultas

In [28]:
# Crear índices
cursor.execute("CREATE INDEX IF NOT EXISTS idx_usuarios_id ON Usuarios(ID)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_pedidos_usuarioid ON Pedidos(UsuarioID)")

### Ejemplo de consulta optimizada con JOINs

In [45]:
# Consulta optimizada con JOINs
print("Consulta optimizada con JOINs:")
for row in cursor.execute('''
SELECT Usuarios.Nombre, Usuarios.Email, Pedidos.ID AS PedidoID
FROM Usuarios
INNER JOIN Pedidos ON Usuarios.ID = Pedidos.UsuarioID;
'''):
    print(row)

Consulta optimizada con JOINs:
('Gus', 'gus@ejemplo.com', 1)
